In [15]:
from __future__ import print_function
import numpy as np
import csv
import time
from datetime import datetime
t1 = datetime.now()

In [16]:
filename = 'data.txt'
output_filename = 'outputp.txt'
with open(filename, 'rb') as f:
    reader = csv.reader(f)
    table = map(tuple, reader)
    table = table

In [17]:
def add(tup1, v):
    return set2tuple(set(tup1) | {v})
    
def remove(tup1, v):
    return set2tuple(set(tup1) - {v})

def inter(tup1, tup2):
    return set2tuple(set(tup1) & set(tup2))

def union(tup1, tup2):
    return set2tuple(set(tup1) | set(tup2))

def diff(tup1, tup2):
    return set2tuple(set(tup1) - set(tup2))

def drop(tup1, index):
    if index < 0:
        index += len(tup1)
    return tup1[0:index] + tup1[index+1:]

def set2tuple(s):
    return tuple(sorted(list(s)))

In [18]:
from collections import defaultdict
partitions = {}
def getTupleFromOne(x):
    return (x,)
def get_partition(attributes):
    if attributes in partitions:
        return partitions[attributes]
    
    if len(attributes) == 0:
        partitions[attributes] = []
    elif len(attributes) == 1:
        iAttr = attributes[0]
        d = defaultdict(tuple)
        for index, row in enumerate(table):
            d[row[iAttr]] += (index, )
        partitions[attributes] = d.values()
    else:
        ps1 = get_partition(attributes[0:-1])
        ps2 = get_partition(attributes[0:-2] + attributes[-1:])
        s = []
        iRow2p = {}
        for i, p1 in enumerate(ps1):
            for iRow in p1:
                iRow2p[iRow] = i

        for p2 in ps2:
            tmp = defaultdict(tuple)
            for iRow in p2:
                tmp[iRow2p[iRow]] += (iRow, )
            s += tmp.values()
        partitions[attributes] = s
        
    return partitions[attributes]

In [19]:
def isValid(X, E): 
    '''
    test if X\{E} -> E is valid
    X is a set of number, E is a number
    '''
    return len(get_partition(set2tuple(X - {E}))) == len(get_partition(set2tuple(X)))
    
def generate_next_level(L):
    # list comprehension?
    Ln = set([])
    for l1 in L:
        for l2 in L:
            if l1 != l2 and len(diff(l1, l2)) == 1:
                Ln.add(union(l1, l2))
    return Ln

def mycmp(fd1, fd2):
    if list(fd1[0]) < list(fd2[0]) or (fd1[0] == fd2[0] and fd1[1] < fd2[1]):
        return -1
    elif list(fd1[0]) > list(fd2[0]) or (fd1[0] == fd2[0] and fd1[1] > fd2[1]):
        return 1
    else:
        return 0

def output_fd(fds):
    with open(output_filename, 'w') as f:
        fds_sorted = sorted(fds, cmp=mycmp)
        for fd in fds_sorted:
            str = ''
            for l in fd[0]:
                str += '%d ' % (l+1)
            str += '-> %d\n' % fd[1]
            f.write(str)
            # print(str, end='')

In [20]:
# RHS is a map from "tuple of number" to "set of number"
maxL = len(table[0])
R = set(range(maxL))
RHS = {(): R}
fds = []
def compute_dependencies(L): # L is a set of tuple of number
    L_new = L.copy()
    for X in L:
        Xs = set(X)
        # X is a tuple of number like (1, 2, 5, ..., n)
        # Xs: set of number.
        RHS[X] = R
        # E: number
        for E in Xs:
            RHS[X] = RHS[X] & RHS[set2tuple(Xs - {E})]
        for E in RHS[X] & Xs:
            if isValid(Xs, E):
                fds.append((set2tuple(Xs - {E}), E))
                RHS[X].remove(E)
                RHS[X] = RHS[X] & Xs
        if len(RHS[X]) == 0:
            L_new.remove(X)
    
    return L_new

In [21]:
def main():
    # L is a set of tuple of number
    L = set(map(getTupleFromOne, R))
    L = compute_dependencies(L)
    for i in range(1, maxL):
        L = compute_dependencies(generate_next_level(L))

    output_fd(fds)

%prun main()
    
print('time used: %s' %(datetime.now() - t1))

 time used: 0:00:31.617200
